In [2]:
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
import requests as rqs
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy import integrate
import pulp
from IPython.display import clear_output
from time import sleep
from unidecode import unidecode
import math

import warnings
warnings.filterwarnings("ignore")

In [3]:
def single_team(df) :
    if len(df) == 1: # If player played for a single team, do nothing
        return df
    else: # If player has moved, return total stats with last team
        row = df[df['Tm'] == 'TOT'].copy()
        row['Tm'] = [str(df['Tm'].iloc[-1])]
        return row

def multiplier(n, mean, std) :

    if n == 0 : # If there's no game : 0 score prediction
        return 0
    elif n == 1 : # If there's 1 game : no multiplier needed
        return 1
    elif math.isnan(mean) : # If there's 1 game : no multiplier needed
        return 1
    elif math.isnan(std) : # If there's 1 game : no multiplier needed
        return 1

    else : # More than 1 game : multiplier required
        # Define PDF of distribution of Y = max{X1, X2, ..., Xn}, where Xi 〜 N(mean, std^2)
        def max_PDF(y, n, mean, std) :
            f = n * np.exp(- (y - mean)**2 / (2 * std*std)) / np.sqrt(2 * np.pi * std*std) * (0.5 + 0.5 * erf((y - mean) / np.sqrt(2 * std*std))) ** (n - 1)
            return f
        # Calculte the mean of the distribution of the max
        E_max = round(integrate.quad(lambda x: x * max_PDF(x, n, mean, std), -np.inf, np.inf)[0], 3)

        # Calculate the value of the score multiplier
        if mean == 0 :
            mean = 1
            
        return E_max / mean

def SoRareScore(df):

    df = df.apply(pd.to_numeric, errors = 'ignore')
    scores = []
    for pts, reb, ast, blk, stl, tov, fg_3 in zip(df['PTS'], df['TRB'], df['AST'], df['BLK'], df['STL'], df['TOV'], df['3P']) :
        
        score = 0
        if pts not in ['Inactive', 'Did Not Play', 'Did Not Dress', 'Not With Team', 'Player Suspended']:
            
            pts, reb, ast, blk, stl, tov, fg_3 = int(pts), int(reb), int(ast), int(blk), int(stl), int(tov), int(fg_3)

            score = 1 * pts + 1.2 * reb + 1.5 * ast + 3 * blk + 3 * stl + 1 * fg_3 - 2 * tov

            doubles = 0 # Count double-digits stats
            if pts >= 10 :
                doubles += 1
            if reb >= 10 :
                doubles += 1
            if ast >= 10 :
                doubles += 1
            if blk >= 10 :
                doubles += 1
            if stl >= 10 :
                doubles += 1

            if doubles == 2 : # Double-double
                score += 1

            if doubles >= 3 : # Tripe-double
                score += 2
        
        else :
            score = 'dnp'

        scores.append(score)
    return scores

In [4]:
# Define possible competitions
conferences = {'WEST' : ['DEN', 'MEM', 'SAC', 'NOP', 'DAL', 'UTA', 'MIN', 'LAC', 'GSW', 'OKC', 'PHO', 'POR', 'LAL', 'SAS', 'HOU'],
                   'EAST' : ['BOS', 'MIL', 'PHI', 'BRK', 'CLE', 'NYK', 'MIA', 'ATL', 'IND', 'CHI', 'TOR', 'WAS', 'ORL', 'CHO', 'DET']}

common_contender   = { 'NAME'         :  'COMMON CONTENDER',
                       'CAP'          :  110,
                       'MVP'          :  False,
                       'ELIGIBILITY'  :  None}

pickup             = { 'NAME'         :  'PICKUP',
                       'CAP'          :  120,
                       'MVP'          :  True,
                       'ELIGIBILITY'  :  None}

common_all_offense = { 'NAME'         :  'COMMON ALL OFFENSE',
                       'CAP'          :  120,
                       'MVP'          :  True,
                       'ELIGIBILITY'  :  None}    

limited_contender =  { 'NAME'         :  'LIMITED CONTENDER',
                       'CAP'          :  110,
                       'MVP'          :  False,
                       'ELIGIBILITY'  :  None}   

common_u23        =  { 'NAME'         :  'COMMON UNDER 23',
                       'CAP'          :  120,
                       'MVP'          :  True,
                       'ELIGIBILITY'  :  "Age <= 23"} 

common_veterans   =  { 'NAME'         :  'COMMON VETERANS',
                       'CAP'          :  120,
                       'MVP'          :  True,
                       'ELIGIBILITY'  :  "Age >= 30"} 

common_west       =  { 'NAME'         :  'COMMON WESTERN CONFERENCE',
                       'CAP'          :  120,
                       'MVP'          :  True,
                       'ELIGIBILITY'  :  "Tm == @conferences.get('WEST')"}   

common_east       =  { 'NAME'         :  'COMMON EASTERN CONFERENCE',
                       'CAP'          :  120,
                       'MVP'          :  True,
                       'ELIGIBILITY'  :  "Tm == @conferences.get('EAST')"}   

common_underdog   =  { 'NAME'         :  'COMMON UNDERDOG',
                       'CAP'          :  60,
                       'MVP'          :  False,
                       'ELIGIBILITY'  :  None}  

common_nocap      =  { 'NAME'         :  'COMMON NO CAP',
                       'CAP'          :  10000000,
                       'MVP'          :  False,
                       'ELIGIBILITY'  :  None}   



In [5]:
months = ['october', 'november', 'december', 'january', 'february', 'march', 'april']

monthly_schedule_list = []
for month in months:
    url = f'https://www.basketball-reference.com/leagues/NBA_2023_games-{month}.html'
    page = rqs.get(url)
    soup = BeautifulSoup(page.content, 'html')
    while soup.find('tr', class_ = 'thead') is not None :
        soup.find('tr', class_ = 'thead').decompose()

    table = soup.find('table')
    monthly_schedule = pd.read_html(str(table))[0]

    home_tm, away_tm = [], []
    for i, x in enumerate(table.find_all('a', href = True), start = 1):
        if x['href'].split('/')[1] == 'teams' :
            if i % 2 == 1 :
                home_tm.append((x['href'][7:10]))
            else :
                away_tm.append((x['href'][7:10]))

    monthly_schedule = monthly_schedule.assign(home = home_tm, away = away_tm)[['Date', 'home', 'away']]
    monthly_schedule['Date'] = monthly_schedule['Date'].apply(lambda x: datetime.datetime.strptime(x, '%a, %b %d, %Y'))
    monthly_schedule_list.append(monthly_schedule)

schedule = pd.concat(monthly_schedule_list).reset_index(drop = True)

gw = 1
dates, gws = [], []
date = datetime.datetime(2022, 10, 21)
while date < datetime.datetime(2023, 5, 10):
    dates.append(date)
    gws.append(gw)
    if date.weekday() in [3, 6]:
        gw = gw + 1

    date = date + datetime.timedelta(days = 1)

gameweeks = pd.DataFrame(zip(dates, gws), columns = ['Date', 'Gameweek'])
schedule = schedule.merge(gameweeks, on = 'Date')

schedule.head()

,Date,home,away,Gameweek
0,2022-10-21,CHO,NOP,1
1,2022-10-21,IND,SAS,1
2,2022-10-21,WAS,CHI,1
3,2022-10-21,ATL,ORL,1
4,2022-10-21,BRK,TOR,1


In [6]:
# THE MIGHTY FUNCTION
def my_teams(gameweek, competitions, ignore_players, limited = False) :
    
    # INPUT  :  competitions - list of competition dictionaries (in order of importance)
    # competition = {'NAME'  :  , 'CAP'  :  , 'MVP'  :  }

    ### LOAD TRAINGING DATA AND MY PLAYERS ###
    data_train = pd.read_csv('training_data.csv')
    if limited :
        with open('my_players_limited.txt', 'r') as file:
            my_players = file.read().splitlines()
    else :
        with open('my_players.txt', 'r') as file:
            my_players = file.read().splitlines()
    data_train['SORARE'] = data_train['SORARE'].astype(float)
    #############################

    my_players = [player for player in my_players if player not in ignore_players]

    selected_players = []
    ### COLLECT LIVE PLAYER DATA ###
    print('Collecting live player data ...')

    url = f'https://www.basketball-reference.com/leagues/NBA_2023_per_game.html'
    page = rqs.get(url)
    soup = BeautifulSoup(page.content, 'html')
    while soup.find('tr', class_ = 'thead') is not None :
        soup.find('tr', class_ = 'thead').decompose()
    table = soup.find('table')
    data_live = pd.read_html(str(table))[0][['Player', 'Tm', 'Age']]
    hrefs = []
    for x in table.find_all('a', href = True):
        if x['href'].split('/')[1] == 'players' :
            hrefs.append(x['href'][:-5])
    data_live['href'] = hrefs
    data_live = data_live.groupby('Player').apply(single_team)
    data_live = data_live.reset_index(drop = True)
    #############################

    ### CREATE FEATURES FOR MY PLAYERS ###
    clear_output()
    print('Fetching my players game logs ...')

    my_data = data_live[data_live['Player'].isin(my_players)]
    
    if len(my_players) != len(my_data) :
        print('Player name not recognized')
    my_players_info = []
    for i, player, href, age, tm in zip(range(len(my_data)), my_data['Player'], my_data['href'], my_data['Age'], my_data['Tm']) :

        print(f'     ... {round(100*(i+1)/len(my_data), 1)}% {player}                 ', end = '')
        print('\r', end = '')

        ## SCRAPE PLAYER GAME LOGS ##
        url = f'https://www.basketball-reference.com{href}/gamelog/2023'
        page = rqs.get(url)
        soup = BeautifulSoup(page.content, 'html')
        while soup.find('tr', class_ = 'thead') is not None :
            soup.find('tr', class_ = 'thead').decompose()
        table = soup.find('table', id = 'pgl_basic')
        df = pd.read_html(str(table))[0][['Date', 'Tm', 'G', 'MP', 'PTS', 'TRB', 'AST', 'BLK', 'STL', 'TOV', '3P']]
        df.insert(0, 'Player', len(df) * [player])
        df = df.apply(pd.to_numeric, errors = 'ignore')
        df['SORARE'] = SoRareScore(df)
        df['Date'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
        df = df.sort_values('Date')
        df['SORARE'] = df['SORARE'].replace('dnp', np.nan)
        df['SORARE'] = df['SORARE'].astype(float)
        df['SORARE_round'] = df['SORARE'].apply(lambda x: x if np.isnan(x) else round(x))
        #############################

        ## MERGE PLAYER WITH ITS TEAM'S SCHEDULE ##
        if len(set(df['Tm'])) == 1 :
    
            tm = df['Tm'].iloc[-1]
            player_schedule = schedule[(schedule['home'] == tm) | (schedule['away'] == tm)]
        else :

            og_tm = list(df['Tm'])[0]
            player_history = {}
            for tm, d in zip(df['Tm'], df['Date']):
                
                if og_tm != tm:
                    dftm = df.groupby('Tm').get_group(og_tm)
                    player_history[og_tm] = [dftm['Date'].min(), dftm['Date'].max()]

                og_tm = tm
            dftm = df.groupby('Tm').get_group(og_tm)
            player_history[og_tm] = [dftm['Date'].min(), datetime.datetime(2100, 1, 1)]
            player_schedule_tm = []
            for tm, td in player_history.items():
                player_schedule = schedule[(schedule['Date'] >= td[0]) & (schedule['Date'] <= td[1])]
                player_schedule = player_schedule[(schedule['home'] == tm) | (schedule['away'] == tm)]
                player_schedule_tm.append(player_schedule)
            player_schedule = pd.concat(player_schedule_tm).sort_values('Date')
        
        away_games = player_schedule.rename(columns = {'away' : 'Tm'}).merge(df, on = ['Date', 'Tm'], how = 'left')
        home_games = player_schedule.rename(columns = {'home' : 'Tm'}).merge(df, on = ['Date', 'Tm'], how = 'left')
        away_games = away_games[away_games['Tm'] == tm].rename(columns = {'home' : 'opp'})
        home_games = home_games[home_games['Tm'] == tm].rename(columns = {'away' : 'opp'})
        df = pd.concat([home_games, away_games]).sort_values('Date')
            
        df['Rest'] = [np.nan] + [(t1 - t0).days for t1, t0 in zip(df.Date[1:], df.Date[:-1])]
        df = df[df['Gameweek'] <= gameweek]
        df_past = df[df['Player'].notna()]
        df_pred = df[df['Player'].isna()]
        #############################

        ## CREATE MACHINE LEARNING FEATURES ##
        form_1 = df_past[df_past['SORARE'].notna()].SORARE.iloc[-1]
        form_2 = df_past['SORARE'].tail(2).mean()
        form_5 = df_past['SORARE'].tail(5).mean()
        form_10 = df_past['SORARE'].tail(10).mean()
        cost = df_past[(df_past['SORARE'].notna()) & (df_past['Gameweek'] < gameweek-1)].tail(10).SORARE.mean()
        form_25 = df_past['SORARE'].tail(25).mean()
        rest = df_pred.Rest.mean()
        stdev = df_past['SORARE'].std()
        features = ['form_1', 'form_2', 'form_5', 'form_10', 'mean_score', 'Rest']
        player_form = pd.DataFrame([[player, age, tm, round(cost), stdev, gameweek, list(df_pred['opp']), len(df_pred), form_1, form_2, form_5, form_10, form_25, rest]],
                                                columns = ['Player', 'Age', 'Tm', 'Cost', 'Std.', 'Gameweek', 'Against', 'No. of games'] + features)
        my_players_info.append(player_form)
        sleep(2)
        
    my_data = pd.concat(my_players_info).reset_index(drop = True)
    my_data['mean_score'] = my_data['mean_score'].replace(np.nan, 0)
    my_data['Rest'] = my_data['Rest'].replace(np.nan, 1.0)
    for w in [1, 2, 5, 10] :
        my_data[f'form_{w}'] = my_data[f'form_{w}'].fillna(my_data['mean_score'])
    #############################

    ### INJURIES ###
    injury_report = 'INJURY REPORT :'
    url = 'https://www.espn.co.uk/nba/injuries'
    page = rqs.get(url)
    soup = BeautifulSoup(page.content, 'html')
    tables = soup.find_all('table')
    injuries = pd.concat(pd.read_html(str(tables))).reset_index(drop = True)

    my_data['Name_merge'] = my_data['Player'].apply(lambda x: unidecode(x).lower().replace('.', '').replace('jr', '').replace('sr', '').replace('ii', '').replace('iii', ''))
    injuries['Name_merge'] = injuries['NAME'].apply(lambda x: unidecode(x).lower().replace('.', '').replace('jr', '').replace('sr', '').replace('ii', '').replace('iii', ''))
    my_data = my_data.merge(injuries, on = 'Name_merge', how = 'left')
    my_data['STATUS'] = my_data['STATUS'].replace(np.nan, 'Fit')
    my_injuries = my_data[my_data['STATUS'] != 'Fit']
    my_data = my_data[my_data['STATUS'] == 'Fit']
    for player, status in zip(my_injuries['NAME'], my_injuries['STATUS']) :
        injury_report = injury_report + f' {player} ({status.upper()});'
    #############################

    ### COMPUTE MY PLAYERS PROJECTED SCORES ###
    clear_output()
    print('Predicting my players scores ... ')

    model = LinearRegression()
    model.fit(data_train[features], data_train['SORARE'])
    my_data = my_data.assign(pred_raw = model.predict(my_data[features]))
    my_data['multiplier'] = [multiplier(n, mean, std) for n, mean, std in zip(my_data['No. of games'], my_data['mean_score'], my_data['Std.'])]
    my_data['pred'] = my_data['multiplier'] * my_data['pred_raw']
    my_data = my_data.sort_values('Cost', ascending = False)
    sleep(2)
    #############################
 
    ### OPTIMISE TOTAL PROJECTED SCORE FOR ALL INPUT COMPETITIONS ###
    clear_output()
    pool = my_data.copy()
    print('Optimising competition lineups ... ')
    for competition in competitions:

        if competition.get('ELIGIBILITY') is not None :
            player_pool = pool.query(competition.get('ELIGIBILITY'))
        else :
            player_pool = pool

        # Initiate maximization problem with PuLP
        prob = pulp.LpProblem('prob', pulp.LpMaximize)
        # Create decision variables : Is the player chosen ?
        x = pulp.LpVariable.dicts('x', player_pool['Player'], lowBound = 0, upBound = 1, cat = pulp.LpInteger)
        var_x = list(x.values())

        # Objective function : Maximize the total predicted score
        prob += pulp.lpSum([x * PRED_SCORE for x, PRED_SCORE in zip(var_x, player_pool['pred'])])
        prob += pulp.lpSum(var_x) == 5

        if competition.get('MVP'):
            # Add MVP decision variables
            y = pulp.LpVariable.dicts('mvp', player_pool['Player'], lowBound = 0, upBound = 1, cat = pulp.LpInteger)
            var_y = list(y.values())
            # Constraints
            prob += pulp.lpSum([(x - y) * COST for x, y, COST in zip(var_x, var_y, player_pool['Cost'])]) <= competition.get('CAP')
            prob += pulp.lpSum(var_y) == 1
            for vx, vy in zip(var_x, var_y) :
                prob += vx - vy >= 0

        else:
            # Budget constraint
            prob += pulp.lpSum([x * COST for x, COST in zip(var_x, player_pool['Cost'])]) <= competition.get('CAP')

        # Solve optimisation problem
        status = prob.solve(pulp.PULP_CBC_CMD(msg = False))
        
        # Save selected players list
        sel_players = []
        for d in  x.items():
            if d[1].varValue == 1.0 :
                sel_players.append(d[0])

        pool = pool[~pool['Player'].isin(sel_players)] # Get rid of already selected players
        selected_players.append(sel_players)

        # Save MVP
        sel_mvp = []
        if competition.get('MVP'):
            for d in  y.items():
                if d[1].varValue == 1.0 :
                    sel_mvp = d[0]

        #############################

    for competition, players in zip(competitions, selected_players) :

        comp_df = my_data[my_data['Player'].isin(players)]
        print(f"\nCompetition : {competition.get('NAME')}, Projected points : {round(comp_df.pred.sum(), 1)}")
        for player, cost, pred, status in zip(comp_df['Player'], comp_df['Cost'], comp_df['pred'], comp_df['STATUS']) :
            if player == sel_mvp :
                player = '*' + player
            else :
                player = ' ' + player
                
            print(f'     {player.upper()} ({cost}) : {round(pred, 1)} pts; {status}')

    print('\n' + injury_report)
    coldisp = ['Player', 'Age', 'Cost', 'pred', 'Against', 'No. of games', 'STATUS', 'COMMENT']

    return my_data[coldisp]

In [7]:
competitions = [pickup, common_veterans]
gameweek = 37

# NEED TO FIX FOR TRADED PLAYERS
ignore_players = ['Cameron Johnson', 'Damian Jones', 'George Hill', 'Jarred Vanderbilt', 'Reggie Jackson', 'Darius Bazley', 
                    'Kevin Durant', 'Jakob Poeltl']
selection = my_teams(gameweek, competitions, ignore_players, limited = False)

Optimising competition lineups ... 

Competition : PICKUP, Projected points : 25.7
      TERRENCE ROSS (28) : 21.2 pts; Fit
      OUSMANE DIENG (11) : 0.0 pts; Fit
      ADMIRAL SCHOFIELD (11) : 0.0 pts; Fit
      DANUEL HOUSE JR. (6) : 0.0 pts; Fit
      KZ OKPALA (2) : 4.6 pts; Fit

Competition : COMMON VETERANS, Projected points : 0.0
      JONAS VALANČIŪNAS (25) : 0.0 pts; Fit
      HARRISON BARNES (20) : 0.0 pts; Fit
      BISMACK BIYOMBO (16) : 0.0 pts; Fit
     *THADDEUS YOUNG (12) : 0.0 pts; Fit
      BOBAN MARJANOVIĆ (6) : 0.0 pts; Fit

INJURY REPORT : Collin Sexton (OUT); Davis Bertans (OUT); Ivica Zubac (DAY-TO-DAY); Jaylen Brown (OUT); Jaylen Nowell (DAY-TO-DAY); Larry Nance Jr. (OUT); Rudy Gobert (DAY-TO-DAY);


In [41]:
url = f'https://www.basketball-reference.com/leagues/NBA_2023_per_game.html'
page = rqs.get(url)
soup = BeautifulSoup(page.content, 'html')
while soup.find('tr', class_ = 'thead') is not None :
    soup.find('tr', class_ = 'thead').decompose()
table = soup.find('table')
data_live = pd.read_html(str(table))[0][['Player', 'Tm', 'Age']]
hrefs = []
for x in table.find_all('a', href = True):
    if x['href'].split('/')[1] == 'players' :
        hrefs.append(x['href'][:-5])
data_live['href'] = hrefs
data_live = data_live.groupby('Player').apply(single_team)
data_live = data_live.reset_index(drop = True)
#############################

In [42]:
my_data = data_live[data_live['Player'].isin(['Kevin Durant'])]


my_players_info = []
for i, player, href, age, tm in zip(range(len(my_data)), my_data['Player'], my_data['href'], my_data['Age'], my_data['Tm']) :

    print(f'     ... {round(100*(i+1)/len(my_data), 1)}% {player}                 ', end = '')
    print('\r', end = '')

    ## SCRAPE PLAYER GAME LOGS ##
    url = f'https://www.basketball-reference.com{href}/gamelog/2023'
    page = rqs.get(url)
    soup = BeautifulSoup(page.content, 'html')
    while soup.find('tr', class_ = 'thead') is not None :
        soup.find('tr', class_ = 'thead').decompose()
    table = soup.find('table', id = 'pgl_basic')
    df = pd.read_html(str(table))[0][['Date', 'Tm', 'G', 'MP', 'PTS', 'TRB', 'AST', 'BLK', 'STL', 'TOV', '3P']]
    df.insert(0, 'Player', len(df) * [player])
    df = df.apply(pd.to_numeric, errors = 'ignore')
    df['SORARE'] = SoRareScore(df)
    df['Date'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df = df.sort_values('Date')
    df['SORARE'] = df['SORARE'].replace('dnp', np.nan)
    df['SORARE'] = df['SORARE'].astype(float)
    df['SORARE_round'] = df['SORARE'].apply(lambda x: x if np.isnan(x) else round(x))
    #############################

 

In [43]:
df

,Player,Date,Tm,G,MP,PTS,TRB,AST,BLK,STL,TOV,3P,SORARE,SORARE_round
0,Kevin Durant,2022-10-19,BRK,1.0,31:46,32,3,2,4,1,4,2,47.6,48.0
1,Kevin Durant,2022-10-21,BRK,2.0,35:27,27,5,6,2,0,5,3,41.0,41.0
2,Kevin Durant,2022-10-24,BRK,3.0,39:01,37,5,4,0,0,1,1,48.0,48.0
3,Kevin Durant,2022-10-26,BRK,4.0,38:52,33,6,3,2,0,5,1,41.7,42.0
4,Kevin Durant,2022-10-27,BRK,5.0,41:14,37,3,5,1,2,3,2,53.1,53.0
5,Kevin Durant,2022-10-29,BRK,6.0,38:47,26,5,2,4,3,3,1,51.0,51.0
6,Kevin Durant,2022-10-31,BRK,7.0,39:04,36,9,7,1,1,3,2,59.3,59.0
7,Kevin Durant,2022-11-01,BRK,8.0,37:59,32,9,6,2,1,7,2,48.8,49.0
8,Kevin Durant,2022-11-04,BRK,9.0,33:11,28,9,11,1,0,2,2,57.3,57.0
9,Kevin Durant,2022-11-05,BRK,10.0,37:20,27,8,2,3,0,3,1,43.6,44.0


In [44]:
og_tm = list(df['Tm'])[0]
player_history = {}
for tm, d in zip(df['Tm'], df['Date']):
    
    if og_tm != tm:
        dftm = df.groupby('Tm').get_group(og_tm)
        player_history[og_tm] = [dftm['Date'].min(), dftm['Date'].max()]


    og_tm = tm
dftm = df.groupby('Tm').get_group(og_tm)
player_history[og_tm] = [dftm['Date'].min(), datetime.datetime(2100, 1, 1)]

In [45]:
player_history

{'BRK': [Timestamp('2022-10-19 00:00:00'), Timestamp('2023-02-07 00:00:00')],
 'PHO': [Timestamp('2023-02-09 00:00:00'),
  datetime.datetime(2100, 1, 1, 0, 0)]}

In [46]:
player_schedule_tm = []
for tm, td in player_history.items():
    print(tm, '', td)
    player_schedule = schedule[(schedule['Date'] >= td[0]) & (schedule['Date'] <= td[1])]
    player_schedule = player_schedule[(schedule['home'] == tm) | (schedule['away'] == tm)]
    player_schedule_tm.append(player_schedule)

BRK  [Timestamp('2022-10-19 00:00:00'), Timestamp('2023-02-07 00:00:00')]
PHO  [Timestamp('2023-02-09 00:00:00'), datetime.datetime(2100, 1, 1, 0, 0)]


In [47]:
player_schedule = pd.concat(player_schedule_tm).sort_values('Date')
player_schedule

,Date,home,away,Gameweek
4,2022-10-21,BRK,TOR,1
32,2022-10-24,MEM,BRK,2
41,2022-10-26,MIL,BRK,2
49,2022-10-27,BRK,DAL,2
66,2022-10-29,BRK,IND,3
...,...,...,...,...
1155,2023-04-02,PHO,OKC,47
1171,2023-04-04,SAS,PHO,48
1184,2023-04-06,PHO,DEN,48
1195,2023-04-07,PHO,LAL,49


In [39]:
away_games = player_schedule.rename(columns = {'away' : 'Tm'}).merge(df, on = ['Date', 'Tm'], how = 'left')
home_games = player_schedule.rename(columns = {'home' : 'Tm'}).merge(df, on = ['Date', 'Tm'], how = 'left')
away_games = away_games[away_games['Tm'] == tm].rename(columns = {'home' : 'opp'})
home_games = home_games[home_games['Tm'] == tm].rename(columns = {'away' : 'opp'})
#df = pd.concat([home_games, away_games]).sort_values('Date')
    
#df['Rest'] = [np.nan] + [(t1 - t0).days for t1, t0 in zip(df.Date[1:], df.Date[:-1])]
#df = df[df['Gameweek'] <= gameweek]
#df_past = df[df['Player'].notna()]
#df_pred = df[df['Player'].isna()]
#############################

In [40]:
away_games

,Date,opp,Tm,Gameweek,Player,G,MP,PTS,TRB,AST,BLK,STL,TOV,3P,SORARE,SORARE_round
52,2023-02-07,BRK,PHO,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,2023-02-09,ATL,PHO,32,Kevin Durant,NaN,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,NaN,NaN
54,2023-02-10,IND,PHO,33,Kevin Durant,NaN,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,NaN,NaN
57,2023-02-24,OKC,PHO,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,2023-02-26,MIL,PHO,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,2023-03-01,CHO,PHO,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,2023-03-03,CHI,PHO,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,2023-03-11,SAC,PHO,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,2023-03-14,MIL,PHO,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,2023-03-19,OKC,PHO,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
home_games_tm = []
for tm, td in player_history.items():

    player_schedule = schedule[(schedule['Date'] >= td[0]) & (schedule['Date'] <= td[1])]
    
    away_games = player_schedule.rename(columns = {'away' : 'Tm'}).merge(df, on = ['Date', 'Tm'], how = 'left')
    home_games = player_schedule.rename(columns = {'home' : 'Tm'}).merge(df, on = ['Date', 'Tm'], how = 'left')
    away_games = away_games[away_games['Tm'] == tm].rename(columns = {'home' : 'opp'})
    home_games = home_games[home_games['Tm'] == tm].rename(columns = {'away' : 'opp'})

    player_schedule_tm.append(player_schedule)

,Date,Tm,away,Gameweek,Player,G,MP,PTS,TRB,AST,BLK,STL,TOV,3P,SORARE,SORARE_round
0,2022-10-21,BRK,TOR,1,Kevin Durant,2.0,35:27,27,5,6,2,0,5,3,41.0,41.0
1,2022-10-24,MEM,BRK,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-26,MIL,BRK,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-10-27,BRK,DAL,2,Kevin Durant,5.0,41:14,37,3,5,1,2,3,2,53.1,53.0
4,2022-10-29,BRK,IND,3,Kevin Durant,6.0,38:47,26,5,2,4,3,3,1,51.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2023-04-02,PHO,OKC,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2023-04-04,SAS,PHO,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,2023-04-06,PHO,DEN,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,2023-04-07,PHO,LAL,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
away_games.head(10)

,Date,home,Tm,Gameweek,Player,G,MP,PTS,TRB,AST,BLK,STL,TOV,3P,SORARE,SORARE_round
0,2022-10-21,BRK,TOR,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-24,MEM,BRK,2,Kevin Durant,3.0,39:01,37,5,4,0,0,1,1,48.0,48.0
2,2022-10-26,MIL,BRK,2,Kevin Durant,4.0,38:52,33,6,3,2,0,5,1,41.7,42.0
3,2022-10-27,BRK,DAL,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-29,BRK,IND,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2022-10-31,BRK,IND,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2022-11-01,BRK,CHI,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2022-11-04,WAS,BRK,5,Kevin Durant,9.0,33:11,28,9,11,1,0,2,2,57.3,57.0
8,2022-11-05,CHO,BRK,5,Kevin Durant,10.0,37:20,27,8,2,3,0,3,1,43.6,44.0
9,2022-11-07,DAL,BRK,6,Kevin Durant,11.0,38:40,26,4,4,1,2,3,1,40.8,41.0
